In [2]:
import numpy as np
import pandas as pd
import itertools


from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### 전처리가 이미 완료된 후의 데이터를 사용
보안상 데이터를 전체 공개 불가

In [6]:
data = pd.read_csv('../data/dataset/CAI_data_003.csv')
df2 = pd.read_csv('../data/dataset/VOC_processing_sentence.csv')

In [12]:
df2

,sentence,processing_sentence
0,고객요청사항안됨상담중자연회복됨,고객 요청사 항 안됨 상담 중 자연 회복됨
1,고객요청사항수신불인터넷연결오류뜸재부팅해봄모텔영업용 일중꼭점검원하여 지사이관함회선단말...,고객 요청사항 수신불인터넷 연결 오류 뜸 재부팅해 봄 모텔 영업용 일 중 꼭 점검 ...
2,고객요청사항수신불회선단말특이사항진단결과기타추가연락처연후방,고객 요청사항 수 선불 회선 단말 특이사항 진단 결과 기타 추가 연락처 연후방
3,수신불 네트워크오류 접수보류,수신 불 네트워크 오류 접수 보류
4,고객요청사항수신불연락후 회선단말특이사항진단결과기타추가연락처인입,고객 요청사항 수신 불 연락 후 회선단 말이 특이 사항 진단 결과 기타 추가 연락처인 입
...,...,...
8661,안심권유고객요청사항화질불량시청중 화면 멈추고 꺼졌다 켜지는현상이 있다고함 연락후방문...,안심 권유 고객 요청사항 화질 불량 시청 중 화면 멈추고 꺼졌다 켜지는 현상이 있다...
8662,리콜 넷플릭스연령인증문의 건 리콜하니 해결했다고하심,리콜 넷플릭스 연령 인증 문의 건 리콜하니 해결했다고 하심
8663,고객요청사항대 수신불전송후 리셋,고객 요청사항 대 수신불전송 후 리셋
8664,안심권유 빠른점검요청 고객요청사항 시청불인터넷 연결 오류 나옴모든장비 재부팅 안내...,안심권 유 빠른 점검 요청 고객 요청사항 시청 불이 인터넷 연결 오류 나 옴 모든 ...


In [11]:
df2['processing_sentence'][1]

'고객 요청사항 수신불인터넷 연결 오류 뜸 재부팅해 봄 모텔 영업용 일 중 꼭 점검 원하여 지 사이 관함 회선 단말 특이사항 진단 결과 기타 추가 연락처'

In [14]:
okt = Okt()

tokenized_doc = okt.pos(df2['processing_sentence'][0])
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('고객', 'Noun'), ('요', 'Modifier'), ('청사', 'Noun'), ('항', 'Noun'), ('안됨', 'Adjective'), ('상담', 'Noun'), ('중', 'Noun'), ('자연', 'Noun'), ('회복', 'Noun'), ('됨', 'Verb')]
추출 : 고객 요 청사 항 안됨 상담 중 자연 회복 됨


In [15]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 9
trigram 다섯개만 출력 : ['고객 청사' '고객 청사 안됨' '상담 자연' '상담 자연 회복' '안됨 상담']


In [17]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode(df2['processing_sentence'][0])
candidate_embeddings = model.encode(candidates)

In [18]:
top_n = 3
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

ValueError: Expected 2D array, got 1D array instead:
array=[ 3.46469246e-02  4.46912199e-02  1.01425946e+00  4.90554469e-03
  1.78474680e-01  1.28673464e-01  3.67628545e-01  8.07006285e-02
 -1.26051009e-01 -6.36618286e-02 -2.08385497e-01  3.46655488e-01
 -1.61893606e-01  1.21188797e-01  1.68295518e-01  6.62353784e-02
 -1.96615443e-01 -3.34212989e-01  1.82527810e-01 -3.03446084e-01
 -9.28727686e-02 -1.53576061e-01  1.98704481e-01 -2.35455289e-01
 -1.59430027e-01 -1.83448583e-01  1.70311362e-01 -3.25120181e-01
 -1.39336318e-01  1.77175432e-01  2.49475166e-01 -1.31505415e-01
  1.27929345e-01 -2.28771657e-01  1.04232468e-01  1.51063651e-01
 -5.02617806e-02  1.88032091e-01 -8.50767270e-02 -6.14974163e-02
  3.94949466e-01 -8.94198753e-03  2.37891197e-01  1.56315893e-01
 -2.50667363e-01 -1.02121465e-01 -3.07776732e-03  2.19947487e-01
 -1.30935729e-01 -1.72307417e-01 -3.79816860e-01 -1.35939687e-01
  2.46205598e-01  2.14578763e-01 -3.04076076e-01  1.02632158e-01
 -7.35845938e-02 -5.12078047e-0

In [19]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None

    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [20]:
sentences = []

stopwords = ['고객','요청사','항','요청사항','안심권유','기타 추가 연락처']

for sentence in df2['processing_sentence']:
    for word in stopwords:
        sentence = sentence.replace(word, '')
        sentence = sentence.lstrip()
    sentences.append(sentence)
df2['sen'] = sentences

In [21]:
df2['sen']

0                                          안됨 상담 중 자연 회복됨
1       수신불인터넷 연결 오류 뜸 재부팅해 봄 모텔 영업용 일 중 꼭 점검 원하여 지 사이...
2                               수 선불 회선 단말 특이사 진단 결과  연후방
3                                      수신 불 네트워크 오류 접수 보류
4                         수신 불 연락 후 회선단 말이 특이 사 진단 결과 인 입
                              ...                        
8661    안심 권유   화질 불량 시청 중 화면 멈추고 꺼졌다 켜지는 현상이 있다고 함 연락...
8662                     리콜 넷플릭스 연령 인증 문의 건 리콜하니 해결했다고 하심
8663                                         대 수신불전송 후 리셋
8664    안심권 유 빠른 점검 요청   시청 불이 인터넷 연결 오류 나 옴 모든 장비 재부팅...
8665    수신 불 연락 후 방문 지니 셋탑 전 원안 들어옴 교체 요청 회선 단말 특이사 진단...
Name: sen, Length: 8666, dtype: object

In [ ]:
abstract_sen = []


for i in range(len(df)) :
    

    try :
        okt = Okt()

        tokenized_doc = okt.pos(df2['sen'][i])
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

        n_gram_range = (2, 3)

        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()

        doc_embedding = model.encode([df2['sen'][i]])
        candidate_embeddings = model.encode(candidates)

        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
        
        
        keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=10)
        
        
        abstract_sen.append(keywords)
        print('----------------------------------------------------------')
        print('원본 : ', df2['sen'][i])
        print('요약 : ', abstract_sen[i])
        print('----------------------------------------------------------')

    except :
        abstract_sen.append(df2['sen'][i])

----------------------------------------------------------
원본 :  안됨 상담 중 자연 회복됨
요약 :  ['자연 회복', '상담 자연', '안됨 상담']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 연결 오류 뜸 재부팅해 봄 모텔 영업용 일 중 꼭 점검 원하여 지 사이 관함 회선 단말 특이사 진단 결과 
요약 :  ['단말 특이 진단', '오류 재부팅 모텔', '원하여 사이 관함']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수 선불 회선 단말 특이사 진단 결과  연후방
요약 :  ['특이 진단', '결과 후방', '선불 회선']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불 네트워크 오류 접수 보류
요약 :  ['네트워크 오류', '수신 네트워크', '접수 보류']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불 연락 후 회선단 말이 특이 사 진단 결과 인 입
요약 :  ['특이 진단 결과', '진단 결과', '선단 특이']
----------------------------------------------------------
---------------------------------

----------------------------------------------------------
원본 :  수신 불 네트워크 오류 최대한 빠른 점검 요청 회선 단말 특이사 진단 결과 
요약 :  ['요청 회선 단말', '점검 요청 회선', '오류 최대한 빠른']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불방 쪽만 안됨 셋탑재 부팅 후 정상
요약 :  ['안됨 탑재', '수신 불방', '탑재 부팅 정상']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 연결 오류 뜸 재부팅해 봄 연락 먼저 회선 단말 특이사 진단 결과 
요약 :  ['진단 결과', '연결 오류 재부팅', '먼저 회선']
----------------------------------------------------------
----------------------------------------------------------
원본 :  가능한 일방 요 분 정도 시청되면 화면 꺼짐 소리 정상 회선 단말 특이사 진단 결과  코로나 미 대상
요약 :  ['진단 결과 코로나', '정도 시청 되면', '화면 꺼짐 소리']
----------------------------------------------------------
----------------------------------------------------------
원본 :  일반 아들 방안 됨 전원 적색점 등재 부팅해도 동일 기량 자 방문 요청함 방문 시간 참고
요약 :  ['기량 방문 요청', '일반 아들 방안', '해도 동일 기량']
--------------

----------------------------------------------------------
원본 :  채널 화질 불량 지직 거림 빠른 방문 요청 근본 해결 코로나 미 대상 회선 단말 특이사 진단 결과 
요약 :  ['화질 불량 거림', '해결 코로나 대상', '빠른 방문']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 연결 오류 단말 구별 불가 주소로 임의 접수회 선 단말 특이사 진단 결과 
요약 :  ['주소 임의 접수', '연결 오류', '접수 단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  화면 안 나옴 음성 정상셋 탑재 부팅해도 동일 방문 시간 참고
요약 :  ['시간 참고', '동일 방문', '음성 정상 탑재']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불 연락 후 전원회 선단 말이 특이 사 진단 결과 인 입
요약 :  ['특이 진단 결과', '수신 연락 전원', '전원 선단 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 작동 불 리모컨에 버튼을 안 눌러도 리모컨에 불이 계속 들어오면서 오작동이 됨은 후방 회선 단말 특이사 진단 결과 
요약 :  ['리모컨 계속 들어오면서', '후방 회선 단말',

----------------------------------------------------------
원본 :  긴급 독촉 심함 차 리셋 후 동일 빠른 방문 원함 업무용 당일 취소 건 있을 경우 우선 방문 원함  리셋 접수 수신 불 리셋 회선 단말 특이사 진단 결과 
요약 :  ['취소 있을 경우', '접수 수신 리셋', '독촉 심함 리셋']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불 인터넷 연결 상태 확인 메시지 접수 중 정상 사용됨 회선 단말 특이사 진단 결과 
요약 :  ['상태 확인', '접수 정상 사용', '메시지 접수 정상']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불 외부 신고함
요약 :  ['외부 고함', '수신 외부 고함', '수신 외부']
----------------------------------------------------------
----------------------------------------------------------
원본 :  일 시시 수신 부을 올인 홈 정상 포트리 셋 리 부팅 안내함
요약 :  ['올인 정상', '시시 수신', '부을 올인']
----------------------------------------------------------
----------------------------------------------------------
원본 :  자연 회복 회선 단말 특이사 진단 결과 
요약 :  ['특이 진단', '자연 회복', '회선 단말']
-------------------------------------------

----------------------------------------------------------
원본 :  시청 불 신 없음 기가 지니 외부 입력 후 동일 기가 지니 리셋 후 사용됨 회선 단말 특이사 진단 결과 
요약 :  ['입력 동일 지니', '회선 단말 특이', '리셋 사용']
----------------------------------------------------------
----------------------------------------------------------
원본 :  일리셋 수신 불 리셋
요약 :  ['수신 리셋', '리셋 수신', '리셋 수신 리셋']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리셋 접수 수신 불
요약 :  ['리셋 접수', '접수 수신', '리셋 접수 수신']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불 연락 후 방문 회선 단말 특이사 진단 결과 
요약 :  ['단말 특이 진단', '방문 회선 단말', '연락 방문']
----------------------------------------------------------
----------------------------------------------------------
원본 :  차 리셋 후 동일 빠른 방문 원함 잦은 고장 불만 꼼꼼하게 점검 요청 지난번 다녀간 엔지니어 말고 기술 뛰어난 엔지니 어 방문 원함 민원 심함  리셋 접수 수 선불 회선 단말 특이사 진단 결과 
요약 :  ['리셋 접수 선불', '기술 뛰어난 지니', '불만 꼼꼼하게 점검']
------------------

...

----------------------------------------------------------
원본 :  일 리셋 접수 네트워크 오류인 확인 불 진단 비정상 적색 포트 리셋
요약 :  ['적색 포트 리셋', '접수 네트워크', '확인 진단']
----------------------------------------------------------
----------------------------------------------------------
원본 :  일시기가 지니 음성인식 안됨 방문 요청
요약 :  ['안됨 방문', '지니 음성인식', '시기 지니 음성인식']
----------------------------------------------------------
----------------------------------------------------------
원본 :  코로나 이 상무 재확인 필   입주로 인 한 티 재연결 회선 단말 특이사 임의회 선 접수 안내 진단 결과 확인 불기 타 추가 연락처
요약 :  ['확인 불기 추가', '입주로 연결', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  임의 시청 불 외부 신고 코로 나무 미리 연락요 문제 주장 회선단 말이 특이 사 진단 결과 
요약 :  ['선단 특이 진단', '미리 연락', '문제 주장 선단']
----------------------------------------------------------
----------------------------------------------------------
원본 :  시청 불 로딩 중 리셋 후 정상 회선 단말 특이사 진단 결과 
요약 :  ['진단 결과', '정상 회선 단말', '단말 특이']
-----------------------

----------------------------------------------------------
원본 :  리셋 접수 시간 내외 티브이 수 신불 회선 단말 특이사 진단 결과 
요약 :  ['내외 티브이', '시간 내외', '리셋 접수 시간']
----------------------------------------------------------
----------------------------------------------------------
원본 :  코로나 이 상무   덕비 수신 불량 신미 약 회선 단말 특이사 카드번호 확인 진단 결과 연
요약 :  ['단말 특이 카드', '덕비 수신', '확인 진단 결과']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수 선불 회선 단말 특이사 진단 결과 
요약 :  ['특이 진단 결과', '선불 회선', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  최대한 빠른 점검요 해지 징 후보임일 시 안내함 시청 중 끊김 문자가 나오면 누르면 되고 해서 교체 요망
요약 :  ['점검 해지', '해서 교체 요망', '나오면 누르면 되고']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리셋 접수 수신 불 리셋 재부팅 회선 단말 특이사 진단 결과 
요약 :  ['수신 리셋 재부팅', '진단 결과', '단말 특이']
---------------------------------------

----------------------------------------------------------
원본 :  리셋 접수 시간 내외 시청 시 끊김 회선 단말 특이사 진단 결과 
요약 :  ['접수 시간', '리셋 접수 시간', '내외 시청 끊김']
----------------------------------------------------------
----------------------------------------------------------
원본 :  임시 스카이 빌 티 끊김 집주인인 이 세입자 번 알려주셔서 시에 세입자분 연락처임 발심해서 안내하니 세입자 번 알려주겠다고 하고 전화 끊김 발신했으나 부응 전송
요약 :  ['했으나 부응 전송', '연락처 심해서 안내', '알려주겠다고 하고 전화']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수 선불 회선 단말 특이사 진단 결과 
요약 :  ['특이 진단 결과', '선불 회선', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 파손 구매 등록 안내
요약 :  ['리모컨 파손', '리모컨 파손 구매', '구매 등록 안내']
----------------------------------------------------------
----------------------------------------------------------
원본 :  차 리셋해도 동일 지연 불만 독촉 심 연락 후 방문요  리셋 접수 시간 내외수 신물 회선 단말 특이사 진단 결과 
요약 :  ['연락 방문 리셋', '해도 동일 지연', '방문 리셋']
-

----------------------------------------------------------
원본 :  리모컨 작동 이상 건전지 재부팅 동일 매장 안내 안내
요약 :  ['동일 매장', '이상 건전지', '건전지 재부팅 동일']
----------------------------------------------------------
----------------------------------------------------------
원본 :  임의회 선 티브이 수신 불 엔지니어 연락 달라고 하심 시설팀이라 한 회 선 단말 특이사 진단 결과 
요약 :  ['티브이 수신', '진단 결과', '연락 달라']
----------------------------------------------------------
----------------------------------------------------------
원본 :  차 리셋 후 동일 빠른 방문 원함  리셋 티브이 수 신불 회선 단말 특이사 진단 결과 
요약 :  ['리셋 동일', '리셋 티브이 회선', '빠른 방문 원함']
----------------------------------------------------------
----------------------------------------------------------
원본 :  코로나 이 상무 재확인 필 리셋 접수 건수 신볼 회선 단말 특이사 진단 결과 정상기 타 추가 연락처
요약 :  ['접수 건수', '상무 확인 리셋', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 작동 불가 매장 거부 무료 배송 등록 안내 회선 단말 특이사 진단 결과 
요약 :  ['작동 불가 매장', '무료 배송 등록', '등록 안내']
----------

----------------------------------------------------------
원본 :  리모컨 작동 부을 매장 방문 안내
요약 :  ['리모컨 작동', '부을 매장', '매장 방문 안내']
----------------------------------------------------------
----------------------------------------------------------
원본 :  독촉 중간에 접수 취소 건 발생 시 당겨서 방문 지원 부탁드립니다   새로 이사 와 서 선 연결 요청하심 리모컨도 작동이 안 된다고 하시고 같이 점검을 요청하십니다 위치 이동 없음 임의 회선으로 접수연 후방 회선 단말 특이사 비즈 형 상품
요약 :  ['당겨서 방문 지원', '리모컨 작동 된다고', '점검 요청 하십니다']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리셋 접수 시간 전후 리콜 안내 일부 채널 수신 불 가입이 필요하다고 나옴 지니 대회 선 단말 특이사 진단 결과 
요약 :  ['리콜 안내', '대회 단말 특이', '필요하다고 나옴 지니']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리셋 접수   안방 수신 불 신 없음이라고 뜬다고 함 전체 리셋 회선 단말 특이사 진단 결과 정상기 타 추가 연락처
요약 :  ['접수 안방 수신', '추가 연락처', '뜬다고 전체 리셋']
----------------------------------------------------------
---------------------------------------------------

In [12]:
df['sentence']

0                                   고객 요청사 항 안됨상담 중 자연회복됨
1       고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 ...
2                고객 요청사항 수신불회선 단말특이사항 진단 결과 기타 추가 연락처 연후방
3                                       수신 불네트워크 오류 접수 보류
4            고객 요청사항 수신불 연락 후회선 단말특이사항 진단 결과 기타 추가 연락처인 입
                              ...                        
8661    안심권유 고객 요청사항 화질 불량 시청 중 화면 멈추고 꺼졌다 켜지는 현상이 있다고...
8662                       리콜넷플릭스 연령인증 문의 건 리콜하니 해결했다고 하심
8663                                   고객 요청사항 대수신불전송 후리셋
8664    안심권유 빠른 점검 요청 고객 요청사항 시청 불인터넷 연결오류나 옴 모든 장비 재부...
8665    고객 요청사항 수신불 연락 후 방문지니 셋 탑 전원안 들어옴 교체 요청회 선 단말특...
Name: sentence, Length: 8666, dtype: object

In [13]:
abstract_sen

[['자연 회복', '상담 자연', '안됨 상담'],
 ['오류 재부팅 모텔', '점검 원하여', '모텔 영업 점검'],
 ['특이 진단 결과', '결과 후방', '회선 단말'],
 ['네트워크 오류', '수신 네트워크', '접수 보류'],
 ['특이 진단 결과', '회선 단말', '단말 특이'],
 ['포트 리셋', '미리 연락', '정상 넷플릭스 안됨'],
 ['단말 특이 진단', '수신 서비스', '시작 회선'],
 ['김현철 채팅', '자체 에서 잡음', '받았으나 소리 난다고'],
 ['진단 결과', '난다 리셋', '따닥거리 소리'],
 ['반복 연락 회선', '불량 시청', '화질 불량 시청'],
 ['특이 진단 결과', '결과 후방', '회선 단말'],
 ['방문 요망 회선', '진단 결과', '빠른 방문'],
 ['포트 리셋', '빠른 방문', '리셋 연락 최대한'],
 ['스트림 에러 회복', '진단 결과', '회선 단말'],
 ['리셋 정상', '안됨 리셋', '안됨 리셋 정상'],
 ['입수 재부팅', '정상 사용', '인터넷 지킴 입수'],
 ['실행 불가', '리셋 회선', '진단 결과'],
 ['회선 단말 특이', '진단 결과', '최대한 빠른'],
 ['안됨 리모컨', '리모컨 계속 매장', '매장 안내'],
 ['수신 불셋', '정상 확인', '수신 불셋 탑재'],
 ['정상 사용', '작동 불셋 탑재', '리모컨 작동 불셋'],
 ['들어옴 의뢰', '원안 들어옴', '원안 들어옴 의뢰'],
 ['들어옴 의뢰', '원안 들어옴', '원안 들어옴 의뢰'],
 ['단말 특이 시험', '회선 단말', '코로나 대상'],
 ['탑재 부팅', '해도 동일 보류', '안됨신 없음 외부'],
 ['진단 결과', '요청 단말 특이', '되도록 오전'],
 ['끊어짐 코로나', '회선 단말 특이', '수신 평소 화면'],
 ['연결 오류 재부팅', '동일 방문', '연결 오류'],
 ['회선 단말', '진단 결과', '정상 사용'],
 ['외부

## Max Sum Similarity
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.